<a href="https://colab.research.google.com/github/israelruiz2005/projeto_airbnb/blob/main/Analisando_os_Dados_do_Airbnb_(Salvador_Bahia).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



#### **Data Science na Prática 2.0**
*by [sigmoidal.ai](https://sigmoidal.ai)*

---

# Análise dos Dados do Airbnb - Salvador Bahia

O [Airbnb](https://www.airbnb.com.br/) já é considerado como sendo a **maior empresa hoteleira da atualidade**. Ah, o detalhe é que ele **não possui nenhum hotel**!

Conectando pessoas que querem viajar (e se hospedar) com anfitriões que querem alugar seus imóveis de maneira prática, o Airbnb fornece uma plataforma inovadora para tornar essa hospedagem alternativa.

No final de 2018, a Startup fundada 10 anos atrás, já havia **hospedado mais de 300 milhões** de pessoas ao redor de todo o mundo, desafiando as redes hoteleiras tradicionais.

Uma das iniciativas do Airbnb é disponibilizar dados do site, para algumas das principais cidades do mundo. Por meio do portal [Inside Airbnb](http://insideairbnb.com/get-the-data.html), é possível baixar uma grande quantidade de dados para desenvolver projetos e soluções de *Data Science*. Infelizmente eles conhecem pouco o Brasil, então não foi possivel usar os dados do site para este estudo, para tanto gerei meu dados usando o site https//my.apify.com, recomendo.

<center><img alt="Analisando Airbnb" width="10%" src="https://www.area360.com.au/wp-content/uploads/2017/09/airbnb-logo.jpg"></center>

**Neste *notebook*, iremos analisar os dados referentes à cidade do Salvador - Bahia, e ver quais insights podem ser extraídos a partir de dados brutos.**

## Obtenção dos Dados

Todos os dados usados aqui foram obtidos a partir do site [Apify](https://apify.com/?utm_source=app).

Para esta análise exploratória inicial, foi baixado o arquivo gerado pela ferramenta deles


In [1]:
# importar os pacotes necessarios
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [3]:
# importar o arquivo para um DataFrame
df = pd.read_csv("/content/dataset_airbnb-scraper_2021-02-13_23-45-56-881.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1693,1983) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Análise dos Dados

Esta etapa tem por objetivo criar uma consciência situacional inicial e permitir um entendimento de como os dados estão estruturados.

**Dicionário das variáveis**

* `id` - número de id gerado para identificar o imóvel
* `name` - nome da propriedade anunciada
* `host_id` - número de id do proprietário (anfitrião) da propriedade
* `host_name` - Nome do anfitrião
* `neighbourhood_group` - esta coluna não contém nenhum valor válido
* `neighbourhood` - nome do bairro
* `latitude` - coordenada da latitude da propriedade
* `longitude` - coordenada da longitude da propriedade
* `room_type` - informa o tipo de quarto que é oferecido
* `price` - preço para alugar o imóvel
* `minimum_nights` - quantidade mínima de noites para reservar
* `number_of_reviews` - número de reviews que a propriedade possui
* `last_review` - data do último review
* `reviews_per_month` - quantidade de reviews por mês
* `calculated_host_listings_count` - quantidade de imóveis do mesmo anfitrião
* `availability_365` - número de dias de disponibilidade dentro de 365 dias

Antes de iniciar qualquer análise, vamos verificar a cara do nosso *dataset*, analisando as 5 primeiras entradas.

In [13]:
df2 = df['address']="Salvador"
df


,address,location/lat,location/lng,name,numberOfGuests,pricing/rate/amount,pricing/rate/amount_formatted,pricing/rate/currency,pricing/rate/is_micros_accuracy,pricing/rate_type,pricing/rate_with_service_fee/amount,pricing/rate_with_service_fee/amount_formatted,pricing/rate_with_service_fee/currency,pricing/rate_with_service_fee/is_micros_accuracy,reviews/0/author/firstName,reviews/0/author/hasProfilePic,reviews/0/author/id,reviews/0/author/pictureUrl,reviews/0/author/smartName,reviews/0/author/thumbnailUrl,reviews/0/authorId,reviews/0/canBeEdited,reviews/0/collectionTag,reviews/0/comments,reviews/0/createdAt,reviews/0/id,reviews/0/idStr,reviews/0/language,reviews/0/listing/id,reviews/0/listing/listingIdStr,reviews/0/listing/name,reviews/0/listingId,reviews/0/rating,reviews/0/recipient/firstName,reviews/0/recipient/hasProfilePic,reviews/0/recipient/id,reviews/0/recipient/pictureUrl,reviews/0/recipient/smartName,reviews/0/recipient/thumbnailUrl,reviews/0/recipientId,...,reviews/67/idStr,reviews/67/language,reviews/67/listing/id,reviews/67/listing/listingIdStr,reviews/67/listing/name,reviews/67/listingId,reviews/67/rating,reviews/67/recipient/firstName,reviews/67/recipient/hasProfilePic,reviews/67/recipient/id,reviews/67/recipient/pictureUrl,reviews/67/recipient/smartName,reviews/67/recipient/thumbnailUrl,reviews/67/recipientId,reviews/67/response,reviews/67/role,reviews/67/userFlag,reviews/68/author/firstName,reviews/68/author/hasProfilePic,reviews/68/author/id,reviews/68/author/pictureUrl,reviews/68/author/smartName,reviews/68/author/thumbnailUrl,reviews/68/authorId,reviews/68/canBeEdited,reviews/68/collectionTag,reviews/68/comments,reviews/68/createdAt,reviews/68/id,reviews/68/idStr,reviews/68/listing/id,reviews/68/listing/listingIdStr,reviews/68/listing/name,reviews/68/listingId,reviews/68/rating,reviews/68/recipient/firstName,reviews/68/recipientId,roomType,stars,url
0,Salvador,-12.74386,-38.61347,Excelente Casa - Madre de Deus,6,37,$37,USD,False,nightly,37,$37,USD,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Casa particular,NaN,https://www.airbnb.com/rooms/47781696
1,Salvador,-12.74454,-38.61758,Quarto com Cama de casal na Mini-Quinta Guesth...,3,48,$48,USD,False,nightly,48,$48,USD,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Private room in guest suite,NaN,https://www.airbnb.com/rooms/41926796
2,Salvador,-12.74822,-38.61645,Casa Orla Madre de Deus - Temporada,8,87,$87,USD,False,nightly,87,$87,USD,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Entire house,NaN,https://www.airbnb.com/rooms/47712051
3,Salvador,-12.74313,-38.61630,Casa inteira em Madre de Deus,10,93,$93,USD,False,nightly,93,$93,USD,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Entire house,NaN,https://www.airbnb.com/rooms/41332854
4,Salvador,-12.74471,-38.61718,Apartamento 1 quarto mobiliado em Madre de Deus,4,19,$19,USD,False,nightly,19,$19,USD,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Entire ap

In [4]:
# mostrar as 5 primeiras entradas
df.head()

,address,location/lat,location/lng,name,numberOfGuests,pricing/rate/amount,pricing/rate/amount_formatted,pricing/rate/currency,pricing/rate/is_micros_accuracy,pricing/rate_type,pricing/rate_with_service_fee/amount,pricing/rate_with_service_fee/amount_formatted,pricing/rate_with_service_fee/currency,pricing/rate_with_service_fee/is_micros_accuracy,reviews/0/author/firstName,reviews/0/author/hasProfilePic,reviews/0/author/id,reviews/0/author/pictureUrl,reviews/0/author/smartName,reviews/0/author/thumbnailUrl,reviews/0/authorId,reviews/0/canBeEdited,reviews/0/collectionTag,reviews/0/comments,reviews/0/createdAt,reviews/0/id,reviews/0/idStr,reviews/0/language,reviews/0/listing/id,reviews/0/listing/listingIdStr,reviews/0/listing/name,reviews/0/listingId,reviews/0/rating,reviews/0/recipient/firstName,reviews/0/recipient/hasProfilePic,reviews/0/recipient/id,reviews/0/recipient/pictureUrl,reviews/0/recipient/smartName,reviews/0/recipient/thumbnailUrl,reviews/0/recipientId,...,reviews/67/idStr,reviews/67/language,reviews/67/listing/id,reviews/67/listing/listingIdStr,reviews/67/listing/name,reviews/67/listingId,reviews/67/rating,reviews/67/recipient/firstName,reviews/67/recipient/hasProfilePic,reviews/67/recipient/id,reviews/67/recipient/pictureUrl,reviews/67/recipient/smartName,reviews/67/recipient/thumbnailUrl,reviews/67/recipientId,reviews/67/response,reviews/67/role,reviews/67/userFlag,reviews/68/author/firstName,reviews/68/author/hasProfilePic,reviews/68/author/id,reviews/68/author/pictureUrl,reviews/68/author/smartName,reviews/68/author/thumbnailUrl,reviews/68/authorId,reviews/68/canBeEdited,reviews/68/collectionTag,reviews/68/comments,reviews/68/createdAt,reviews/68/id,reviews/68/idStr,reviews/68/listing/id,reviews/68/listing/listingIdStr,reviews/68/listing/name,reviews/68/listingId,reviews/68/rating,reviews/68/recipient/firstName,reviews/68/recipientId,roomType,stars,url
0,"Madre de Deus, Bahia, Brazil",-12.74386,-38.61347,Excelente Casa - Madre de Deus,6,37,$37,USD,False,nightly,37,$37,USD,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Casa particular,NaN,https://www.airbnb.com/rooms/47781696
1,"Madre de Deus, Bahia, Brazil",-12.74454,-38.61758,Quarto com Cama de casal na Mini-Quinta Guesth...,3,48,$48,USD,False,nightly,48,$48,USD,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Private room in guest suite,NaN,https://www.airbnb.com/rooms/41926796
2,"Centro, Bahia, Brazil",-12.74822,-38.61645,Casa Orla Madre de Deus - Temporada,8,87,$87,USD,False,nightly,87,$87,USD,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Entire house,NaN,https://www.airbnb.com/rooms/47712051
3,"Bahia, Brazil",-12.74313,-38.61630,Casa inteira em Madre de Deus,10,93,$93,USD,False,nightly,93,$93,USD,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Entire house,NaN,https://www.airbnb.com/rooms/41332854
4,"Madre de Deus, Bahia, Brazil",-12.74471,-38.61718,Apartamento 1 quarto mobiliado em Madre de Deus,4,19,$19,USD,False,nightly,19,$19,USD,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N